# Speedy Compression Tutorial

In [ ]:
# Start with imports 

%matplotlib inline

import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score

import plotting
import matplotlib.pyplot as plt

from matplotlib.lines import Line2D
from matplotlib.legend import Legend

from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu

import hls4ml

In [ ]:
# Set the seed and import tensorflow

seed = 0

np.random.seed(seed)

import tensorflow as tf

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
from callbacks import all_callbacks

tf.random.set_seed(seed)

## The Dataset: fetch the jet tagging dataset and preprocess

We are going to classify jets to five different classes.

In [ ]:
# Fetch the dataset
data = fetch_openml('hls4ml_lhc_jets_hlf')

In [ ]:
# Get the inputs X and targets y
X, y = data['data'], data['target']

In [ ]:
# One-Hot encode the targets, e.g. "w" -> [0,0,1,0,0]
le = LabelEncoder()
y = le.fit_transform(y)
y = to_categorical(y, 5)

In [ ]:
# Split the dataset: train and validation
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Preprocess the dataset
scaler = StandardScaler()
X_train_val = scaler.fit_transform(X_train_val)
X_test = scaler.transform(X_test)

In [ ]:
# Save the data as numpy arrays so you can just load it in case you need to restart
np.save('X_train_val.npy', X_train_val)
np.save('X_test.npy', X_test)
np.save('y_train_val.npy', y_train_val)
np.save('y_test.npy', y_test)
np.save('classes.npy', le.classes_)

In [ ]:
# If you have to restart the notebook use these lines insead of fetching the dataset:

# X_train_val = np.load('X_train_val.npy')
# X_test = np.load('X_test.npy')
# y_train_val = np.load('y_train_val.npy')
# y_test = np.load('y_test.npy')
# classes = np.load('classes.npy', allow_pickle=True)

## The basline model: define the graph and train the baseline
We'll use 3 hidden layers with 64, then 32, then 32 neurons. Each layer will use `relu` activation.
Add an output layer with 5 neurons (one for each class), then finish with Softmax activation.

In [ ]:
model_base = Sequential()
model_base.add(Dense(64, input_shape=(16,), name='fc1', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model_base.add(Activation(activation='relu', name='relu1'))
model_base.add(Dense(32, name='fc2', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model_base.add(Activation(activation='relu', name='relu2'))
model_base.add(Dense(32, name='fc3', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model_base.add(Activation(activation='relu', name='relu3'))
model_base.add(Dense(5, name='output', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model_base.add(Activation(activation='softmax', name='softmax'))

Train the model: we'll use Adam optimizer with categorical crossentropy loss. The callbacks will decay the learning rate and save the model into a directory `model_base`. The model isn't very complex, so this should just take a few minutes even on the CPU.

In [ ]:
def train_model(model, name, y):
    # Optimizer:
    adam = Adam(learning_rate=0.0001)

    # Compile the model, use crossentropy as a loss
    model.compile(
        optimizer=adam,
        loss=['categorical_crossentropy'],
        metrics=['accuracy'])

    # Define callbacks
    callbacks = all_callbacks(
        stop_patience=1000,
        lr_factor=0.5,
        lr_patience=10,
        lr_epsilon=0.000001,
        lr_cooldown=2,
        lr_minimum=0.0000001,
        outputDir=name,
    )

    # Fit the model
    model.fit(
        X_train_val,
        y,
        batch_size=1024,
        epochs=30,
        validation_split=0.25,
        shuffle=True,
        callbacks=callbacks.callbacks,
        verbose=2,
    )

In [ ]:
train_model(model_base, name="model_base", y=y_train_val)

### Checking the performance of the baseline

In [ ]:
def evaluation(model1, name1, model2=None, name2=None):
    y_keras = model1.predict(X_test, verbose=0)
    print(f"Accuracy {name1}: {accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras, axis=1))}")
    
    if model2:
        try:
            y_keras2 = model2.predict(X_test, verbose=0)
        except:
            y_keras2 = model2.predict(np.ascontiguousarray(X_test))
        print(f"Accuracy {name2}: {accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_keras2, axis=1))}")       

    # Plot the ROC Curve
    fig, ax = plt.subplots(figsize=(9, 9))
    _ = plotting.makeRoc(y_test, y_keras, le.classes_)
    
    if model2:
        plt.gca().set_prop_cycle(None)  # reset the colors
        _ = plotting.makeRoc(y_test, y_keras2, le.classes_, linestyle='--')

        lines = [Line2D([0], [0], ls='-'), Line2D([0], [0], ls='--')]
        leg = Legend(ax, lines, labels=[name1, name2], loc='lower right', frameon=False)
        ax.add_artist(leg)

In [ ]:
evaluation(model_base, "Keras fp64")

## Knowledge Distillation: train a smaller model

We are going to experiment with knowledge distillation. We will train a smaller directly and using the labels generated by the baseline (a **student**).

In [ ]:
# Prepare soft labels

y_teacher = model_base.predict(X_train_val, verbose=0)

In [ ]:
# Define a smaller model

model_smaller = Sequential()
model_smaller.add(Dense(16, input_shape=(16,), name='fc1', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model_smaller.add(Activation(activation='relu', name='relu1'))
model_smaller.add(Dense(8, name='fc2', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model_smaller.add(Activation(activation='relu', name='relu2'))
model_smaller.add(Dense(5, name='output', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model_smaller.add(Activation(activation='softmax', name='softmax'))

train_model(model_smaller, name="model_smaller", y=y_train_val)

In [ ]:
# Define the student model (same architecture as the smaller model)

model_student = Sequential()
model_student.add(Dense(16, input_shape=(16,), name='fc1', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model_student.add(Activation(activation='relu', name='relu1'))
model_student.add(Dense(8, name='fc2', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model_student.add(Activation(activation='relu', name='relu2'))
model_student.add(Dense(5, name='output', kernel_initializer='lecun_uniform', kernel_regularizer=l1(0.0001)))
model_student.add(Activation(activation='softmax', name='softmax'))

# NOTICE: the only difference with the above is the target label (y)
train_model(model_student, name="model_student", y=y_teacher)

In [ ]:
# Evaluate the two approaches

evaluation(model_smaller, "Without KD", model_student, "With KD")

## Compression

https://github.com/google/qkeras

This time we're going to use QKeras layers. QKeras is "Quantized Keras" for deep heterogeneous quantization of ML models. It is maintained by Google and we recently added support for QKeras model to hls4ml.

We're using `QDense` layer instead of `Dense`, and `QActivation` instead of `Activation`. We're also specifying `kernel_quantizer = quantized_bits(6,0,0)`. This will use 6-bits (of which 0 are integer) for the weights. We also use the same quantization for the biases, and `quantized_relu(6)` for 6-bit ReLU activations.

In [ ]:
qmodel = Sequential()
qmodel.add(
    QDense(
        16,
        input_shape=(16,),
        name='fc1',
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
    )
)
qmodel.add(QActivation(activation=quantized_relu(6), name='relu1'))
qmodel.add(
    QDense(
        8,
        name='fc2',
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
    )
)
qmodel.add(QActivation(activation=quantized_relu(6), name='relu2'))
qmodel.add(
    QDense(
        5,
        name='output',
        kernel_quantizer=quantized_bits(6, 0, alpha=1),
        bias_quantizer=quantized_bits(6, 0, alpha=1),
        kernel_initializer='lecun_uniform',
        kernel_regularizer=l1(0.0001),
    )
)
qmodel.add(Activation(activation='softmax', name='softmax'))

In [ ]:
train_model(qmodel, name="qmodel", y=y_teacher)

In [ ]:
evaluation(model_student, "fp64 with KD", qmodel , "qmodel with KD")

## Convert the model to FPGA firmware with hls4ml

Now we will go through the steps to convert the model we trained to a low-latency optimized FPGA firmware with hls4ml. First, we will evaluate its classification performance to make sure we haven't lost accuracy using the fixed-point data types.The hls4ml Neural Network inference library is controlled through a configuration dictionary. In this example we'll use the most simple variation.

In [ ]:
config = hls4ml.utils.config_from_keras_model(qmodel, granularity='name')

# Necessary fo softmax layer
config['LayerName']['softmax']['exp_table_t'] = 'ap_fixed<18,8>'
config['LayerName']['softmax']['inv_table_t'] = 'ap_fixed<18,4>'

print("-----------------------------------")
plotting.print_dict(config)
print("-----------------------------------")

hls_model = hls4ml.converters.convert_from_keras_model(
    qmodel,
    hls_config=config,
    output_dir='model_cpp/project',
    part='xcu250-figd2104-2L-e' # You can specify different part if necessary
)

In [ ]:
# Compile the model

hls_model.compile()

Let's visualise what we created. The model architecture is shown, annotated with the shape and data types. Note that the default precision is `ap_fixed<16, 6>`, e.g. look at the `output` types. However the types for weight, bias and relu output are inherited from QKeras.

*Advanced config*. You could change a type of the output in the config, for instance
```config['LayerName']['fc1']['result'] = 'ap_fixed<10,2>'```
It is a necessary step to assure accuracy or limit resource usage.

In [ ]:
# Show the graph

hls4ml.utils.plot_model(hls_model, show_shapes=True, show_precision=True, to_file=None)

Conversion was easy! Now let's see how the performance compares to Keras:

In [ ]:
evaluation(qmodel, "QKeras",  hls_model, "hls4ml")

Let's take a look what files are generated. Please take time to take a look at these files.

In [ ]:
! ls model_cpp/project/firmware/{*.h,*.cpp}

### Synthesize

Finally, you can actually use Vivado HLS to synthesize the model. We can run the build using hls4ml API that simply calls `vivado_hls` or go to the terminal and run the synthesis directly. **`vivado_hls` needs to on PATH**.

In [ ]:
hls_model.build(csim=False)